# Getting the code

In [13]:
PROJECTROOT=$(pwd)
# check current system release code name
lsb_release -c

Codename:	jammy


In [14]:
PROGRAM=cmatrix
pull-lp-source $PROGRAM jammy

Traceback (most recent call last):
  File "/usr/bin/pull-lp-source", line 11, in <module>
    PullPkg.main(distro='ubuntu', pull='source')
  File "/usr/lib/python3/dist-packages/ubuntutools/pullpkg.py", line 107, in main
    cls(*args, **kwargs).pull()
  File "/usr/lib/python3/dist-packages/ubuntutools/pullpkg.py", line 400, in pull
    params = self._get_params(options)
  File "/usr/lib/python3/dist-packages/ubuntutools/pullpkg.py", line 318, in _get_params
    (r, v, p) = self.parse_release_and_version(distro, options['release'],
  File "/usr/lib/python3/dist-packages/ubuntutools/pullpkg.py", line 267, in parse_release_and_version
    (release, pocket) = self.parse_release(distro, release)
  File "/usr/lib/python3/dist-packages/ubuntutools/pullpkg.py", line 254, in parse_release
    d = Distribution(distro)
  File "/usr/lib/python3/dist-packages/ubuntutools/lp/lpapicache.py", line 137, in __new__
    if isinstance(data, str) and data.startswith(str(Launchpad._root_uri)):
  File "/usr

: 1

#### There's a network issue. Try to use `apt source <package>`

In [27]:
apt source $PROGRAM | tail # suppress some output



Get:1 http://ftp.sjtu.edu.cn/ubuntu jammy/universe cmatrix 2.0-3 (dsc) [1,901 B]
Get:2 http://ftp.sjtu.edu.cn/ubuntu jammy/universe cmatrix 2.0-3 (tar) [206 kB]
Get:3 http://ftp.sjtu.edu.cn/ubuntu jammy/universe cmatrix 2.0-3 (diff) [4,436 B]
dpkg-source: info: extracting cmatrix in cmatrix-2.0
dpkg-source: info: unpacking cmatrix_2.0.orig.tar.gz
dpkg-source: info: unpacking cmatrix_2.0-3.debian.tar.xz
dpkg-source: info: using patch list from debian/patches/series
dpkg-source: info: applying 0001-cmatrix.1-Improvement-on-man-page.patch
dpkg-source: info: applying 0002-Fix-78.patch
Fetched 212 kB in 1s (291 kB/s)


In [28]:
ls

cmatrix-2.0                  cmatrix_2.0-3.dsc        do_homework.ipynb
cmatrix_2.0-3.debian.tar.xz  cmatrix_2.0.orig.tar.gz


# Create a new patch through `quilt`

Add the follow file into quilt patch:
- CMakeLists.txt
- Makefile.am
- debian/cmatrix.postinst
- testing.sh  

Those are the files that will be modified/added

In [29]:
cd cmatrix-2.0
quilt new "0003-take-home-test.patch"
quilt add "CMakeLists.txt" "Makefile.am" "debian/cmatrix.postinst" "testing.sh" # add
quilt files # check
cd $PROJECTDIR

Patch 0003-take-home-test.patch is now on top
File CMakeLists.txt added to patch 0003-take-home-test.patch
File Makefile.am added to patch 0003-take-home-test.patch
File debian/cmatrix.postinst added to patch 0003-take-home-test.patch
File testing.sh added to patch 0003-take-home-test.patch
CMakeLists.txt
Makefile.am
debian/cmatrix.postinst
testing.sh


# Adding `testing.sh` to source code

In [30]:
MY_SCRIPT="cmatrix-2.0/testing.sh"

cat > "$MY_SCRIPT" <<"EOF"
#!/bin/bash -

MESSAGE="this is a test from Clymber Loong"

if [ "$1" == "stdout" ] ; then
    echo "$MESSAGE" >&1
else
    echo "$MESSAGE" >&2
fi

exit $?
EOF

chmod +x "$MY_SCRIPT"

In [31]:
/bin/bash $MY_SCRIPT        # echo to stderr
/bin/bash $MY_SCRIPT stdout # echo to stdout

this is a test from Clymber Loong
this is a test from Clymber Loong


# Adding `testing.sh` to building and installation procedure

In [32]:
CMAKE_LISTS='cmatrix-2.0/CMakeLists.txt' # used by cmake
AUTOTOOLCFG='cmatrix-2.0/Makefile.am' # used by autoreconf

echo 'install(PROGRAMS testing.sh DESTINATION bin)' >> "$CMAKE_LISTS"
echo 'bin_SCRIPTS = testing.sh' >> "$AUTOTOOLCFG"

# Add package maintainer scripts

In [33]:
POSTINST='cmatrix-2.0/debian/cmatrix.postinst'

cat > "$POSTINST" <<"EOF"
#!/bin/bash -
set -e

if [ "$1" == "configure" ]; then
    testing.sh stdout
fi

#DEBHELPER#
EOF

chmod +x "$POSTINST"

# Save changes into quilt patch

In [34]:
cd cmatrix-2.0
quilt diff 

--- a/CMakeLists.txt
+++ b/CMakeLists.txt
@@ -71,3 +71,4 @@ if     (UNIX)
 	endforeach ()
 endif  ()
 
+install(PROGRAMS testing.sh DESTINATION bin)
--- a/Makefile.am
+++ b/Makefile.am
@@ -48,3 +48,4 @@ install-data-local:
 		echo "Not installing matrix fonts"; \
 	fi
 
+bin_SCRIPTS = testing.sh
--- /dev/null
+++ b/debian/cmatrix.postinst
@@ -0,0 +1,8 @@
+#!/bin/bash -
+set -e
+
+if [ "$1" == "configure" ]; then
+    testing.sh stdout
+fi
+
+#DEBHELPER#
--- /dev/null
+++ b/testing.sh
@@ -0,0 +1,11 @@
+#!/bin/bash -
+
+MESSAGE="this is a test from Clymber Loong"
+
+if [ "$1" == "stdout" ] ; then
+    echo "$MESSAGE" >&1
+else
+    echo "$MESSAGE" >&2
+fi
+
+exit $?


In [35]:
quilt refresh

Refreshed patch 0003-take-home-test.patch


In [ ]:
quilt header --dep3 -e

# Documenting the testing feature

In [ ]:
dch -i

In [36]:
head debian/changelog

cmatrix (2.0-3ubuntu1) jammy; urgency=medium

  * testing.sh: echo a testing message to stderr
  * debian/cmatrix.postinst: echo a testing message to stdout
  * CMakeLists.txt Makefile.am: handle install testing.sh procedure

 -- Clymber Loong <clymberloong@icloud.com>  Tue, 25 Apr 2023 16:17:23 +0800

cmatrix (2.0-3) unstable; urgency=medium



# Testing the feature

Firstly build from the source to get a "xxx.dsc" file

In [37]:
debuild -S -d -us -uc

 dpkg-buildpackage -us -uc -ui -S -d
dpkg-buildpackage: info: source package cmatrix
dpkg-buildpackage: info: source version 2.0-3ubuntu1
dpkg-buildpackage: info: source distribution jammy
dpkg-buildpackage: info: source changed by Clymber Loong <clymberloong@icloud.com>
 dpkg-source --before-build .
 debian/rules clean
dh clean
   dh_clean
 dpkg-source -b .
dpkg-source: warning: Version number suggests Ubuntu changes, but Maintainer: does not have Ubuntu address
dpkg-source: warning: Version number suggests Ubuntu changes, but there is no XSBC-Original-Maintainer field
dpkg-source: info: using source format '3.0 (quilt)'
dpkg-source: info: building cmatrix using existing ./cmatrix_2.0.orig.tar.gz
dpkg-source: info: using patch list from debian/patches/series
patching file CMakeLists.txt
patching file Makefile.am
The next patch would create the file debian/cmatrix.postinst,
which already exists!  Skipping patch.
1 out of 1 hunk ignored
patching file testing.sh
dpkg-source: info: the pa

: 29

## Fix the build issue

*dpkg-source: info: the patch has fuzz which is not allowed, or is malformed*  
*dpkg-source: error: LC_ALL=C patch -t -F 0 -N -p1 -u -V never -E -b -B .pc/0003-take-home-test.patch/ --reject-file=- < cmatrix-2.0.orig.NvKBG8/debian/patches/0003-take-home-test.patch subprocess returned exit status 1*  

I got stuck for a long time by the issue above. I was misled to dive into `quilt` usage. And then I was trying to figure out how to add the `postinst` into the `debian/*` directory. Finally figured out that the real problem is not `quilt` system, it's about the rules of `debian/*` in debian package.

According to dibian policy, https://www.debian.org/doc/debian-policy/, I should not add quilt patches over /debian/ files. The patches are for the code only.

So, my solution is to remove the file `debian/cmatrix.postinst` from the quilt patch '0003-take-home-test.patch'.

In [39]:
# remove the file in quilt patch
quilt remove "debian/cmatrix.postinst"
quilt refresh
# The following command executing on a interactive termial, because jupyter notebook shell can't duel with it.
# quilt --dep3 -e
# git restore debian/cmatrix.postinst

File debian/cmatrix.postinst removed from patch 0003-take-home-test.patch


In [48]:
# rebuild the source
debuild -S -d -us -uc | tail -n 20

dpkg-buildpackage: info: source version 2.0-3ubuntu1
dpkg-buildpackage: info: source distribution jammy
dpkg-buildpackage: info: source changed by Clymber Loong <clymberloong@icloud.com>
 dpkg-source --before-build .
 debian/rules clean
dh clean
   dh_clean
 dpkg-source -b .
dpkg-source: warning: Version number suggests Ubuntu changes, but Maintainer: does not have Ubuntu address
dpkg-source: warning: Version number suggests Ubuntu changes, but there is no XSBC-Original-Maintainer field
dpkg-source: info: using source format '3.0 (quilt)'
dpkg-source: info: building cmatrix using existing ./cmatrix_2.0.orig.tar.gz
dpkg-source: info: using patch list from debian/patches/series
dpkg-source: info: building cmatrix in cmatrix_2.0-3ubuntu1.debian.tar.xz
dpkg-source: info: building cmatrix in cmatrix_2.0-3ubuntu1.dsc
 dpkg-genbuildinfo --build=source -O../cmatrix_2.0-3ubuntu1_source.buildinfo
 dpkg-genchanges --build=source -O../cmatrix_2.0-3ubuntu1_source.changes
dpkg-genchanges: info: not 

In [44]:
# check result
ls -1 ../cmatrix_2.0-3ubuntu1*

../cmatrix_2.0-3ubuntu1.debian.tar.xz
../cmatrix_2.0-3ubuntu1.dsc
../cmatrix_2.0-3ubuntu1_source.build
../cmatrix_2.0-3ubuntu1_source.buildinfo
../cmatrix_2.0-3ubuntu1_source.changes


Bingo! Finally I figured it out. The lesson I learn is that I should learn more about the tools beforehand. A craftsman should master and take care of his tools. Specifically, there's more I need to learn about debian packaging.

## Build .deb package 

In [53]:
pbuilder-dist jammy build ../cmatrix_2.0-3ubuntu1.dsc | tail -n 20

Distribution data outdated. Please check for an update for distro-info-data. See /usr/share/doc/distro-info-data/README.Debian for details.
W: /home/clymber/.pbuilderrc does not exist
ln: failed to create hard link '/home/clymber/pbuilder/aptcache/ubuntu/libmagic-mgc_1%3a5.41-3_amd64.deb' => '/var/cache/pbuilder/build/51514/var/cache/apt/archives/libmagic-mgc_1%3a5.41-3_amd64.deb': Invalid cross-device link
I: new cache content 'bsdextrautils_2.37.2-4ubuntu3_amd64.deb' added
ln: failed to create hard link '/home/clymber/pbuilder/aptcache/ubuntu/bsdextrautils_2.37.2-4ubuntu3_amd64.deb' => '/var/cache/pbuilder/build/51514/var/cache/apt/archives/bsdextrautils_2.37.2-4ubuntu3_amd64.deb': Invalid cross-device link
I: new cache content 'autoconf_2.71-2_all.deb' added
ln: failed to create hard link '/home/clymber/pbuilder/aptcache/ubuntu/autoconf_2.71-2_all.deb' => '/var/cache/pbuilder/build/51514/var/cache/apt/archives/autoconf_2.71-2_all.deb': Invalid cross-device link
I: new cache content 

In [54]:
ls -1 ~/pbuilder/jammy_result/cmatrix*.deb

/home/clymber/pbuilder/jammy_result/cmatrix_2.0-3ubuntu1_amd64.deb
/home/clymber/pbuilder/jammy_result/cmatrix-xfont_2.0-3ubuntu1_all.deb


## Install the package

In [55]:
sudo dpkg -i /home/clymber/pbuilder/jammy_result/cmatrix_2.0-3ubuntu1_amd64.deb

dpkg: warning: downgrading cmatrix from 2.0-3ubuntu3 to 2.0-3ubuntu1
(Reading database ... 226814 files and directories currently installed.)
Preparing to unpack .../cmatrix_2.0-3ubuntu1_amd64.deb ...
Unpacking cmatrix (2.0-3ubuntu1) over (2.0-3ubuntu3) ...
Setting up cmatrix (2.0-3ubuntu1) ...
this is a test from Clymber Loong
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for gnome-menus (3.36.0-1ubuntu3) ...
Processing triggers for desktop-file-utils (0.26-1ubuntu3) ...
Processing triggers for man-db (2.10.2-1) ...


In [56]:
testing.sh

this is a test from Clymber Loong


In [2]:
dpkg -S testing.sh; testing.sh

cmatrix: /usr/bin/testing.sh
this is a test from Clymber Loong


## Rebuild package with sign

In [57]:
debuild -S -d

 dpkg-buildpackage -us -uc -ui -S -d
dpkg-buildpackage: info: source package cmatrix
dpkg-buildpackage: info: source version 2.0-3ubuntu1
dpkg-buildpackage: info: source distribution jammy
dpkg-buildpackage: info: source changed by Clymber Loong <clymberloong@icloud.com>
 dpkg-source --before-build .
 debian/rules clean
dh clean
   dh_clean
 dpkg-source -b .
dpkg-source: warning: Version number suggests Ubuntu changes, but Maintainer: does not have Ubuntu address
dpkg-source: warning: Version number suggests Ubuntu changes, but there is no XSBC-Original-Maintainer field
dpkg-source: info: using source format '3.0 (quilt)'
dpkg-source: info: building cmatrix using existing ./cmatrix_2.0.orig.tar.gz
dpkg-source: info: using patch list from debian/patches/series
dpkg-source: info: building cmatrix in cmatrix_2.0-3ubuntu1.debian.tar.xz
dpkg-source: info: building cmatrix in cmatrix_2.0-3ubuntu1.dsc
 dpkg-genbuildinfo --build=source -O../cmatrix_2.0-3ubuntu1_source.buildinfo
 dpkg-genchange

In [58]:
pbuilder-dist jammy build ../cmatrix_2.0-3ubuntu1.dsc | tail -n 20

Distribution data outdated. Please check for an update for distro-info-data. See /usr/share/doc/distro-info-data/README.Debian for details.
W: /home/clymber/.pbuilderrc does not exist
ln: failed to create hard link '/home/clymber/pbuilder/aptcache/ubuntu/libmagic-mgc_1%3a5.41-3_amd64.deb' => '/var/cache/pbuilder/build/55220/var/cache/apt/archives/libmagic-mgc_1%3a5.41-3_amd64.deb': Invalid cross-device link
I: new cache content 'bsdextrautils_2.37.2-4ubuntu3_amd64.deb' added
ln: failed to create hard link '/home/clymber/pbuilder/aptcache/ubuntu/bsdextrautils_2.37.2-4ubuntu3_amd64.deb' => '/var/cache/pbuilder/build/55220/var/cache/apt/archives/bsdextrautils_2.37.2-4ubuntu3_amd64.deb': Invalid cross-device link
I: new cache content 'autoconf_2.71-2_all.deb' added
ln: failed to create hard link '/home/clymber/pbuilder/aptcache/ubuntu/autoconf_2.71-2_all.deb' => '/var/cache/pbuilder/build/55220/var/cache/apt/archives/autoconf_2.71-2_all.deb': Invalid cross-device link
I: new cache content 

# Generate a "debdiff" file

In [59]:
debdiff ../cmatrix_2.0-3.dsc ../cmatrix_2.0-3ubuntu1.dsc > ../1-2.0-3ubuntu1.debdiff

gpgv: Signature made Wednesday, January 13, 2021 PM01:19:46 HKT
gpgv:                using RSA key 7E7729476D87D6F11D91ACCBC293E7B461825ACE
gpgv: Can't check signature: No public key
dpkg-source: warning: cannot verify signature /home/clymber/GitHub/take-home-work/temp/cmatrix_2.0-3.dsc
gpgv: Signature made Tuesday, April 25, 2023 PM05:34:16 HKT
gpgv:                using RSA key 5E4D8A8E9BD95993E5B291939E0EB9D9B1BB3628
gpgv:                issuer "clymberloong@icloud.com"
gpgv: Can't check signature: No public key
dpkg-source: warning: cannot verify signature /home/clymber/GitHub/take-home-work/temp/cmatrix_2.0-3ubuntu1.dsc


: 1

In [60]:
gpg --keyserver keyserver.ubuntu.com --recv-keys 61825ACE

gpg: key C293E7B461825ACE: public key "Boyuan Yang (Debian Project) <byang@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [61]:
gpg --no-default-keyring -a --export 61825ACE | gpg --no-default-keyring --keyring ~/.gnupg/trustedkeys.gpg --import -

gpg: keybox '/home/clymber/.gnupg/trustedkeys.gpg' created
gpg: key C293E7B461825ACE: public key "Boyuan Yang (Debian Project) <byang@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [63]:
gpg --no-default-keyring -a --export B1BB3628 | gpg --no-default-keyring --keyring ~/.gnupg/trustedkeys.gpg --import -

gpg: key 4518FD6F6008D599: public key "Clymber Loong <clymberloong@icloud.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1
gpg: marginals needed: 3  completes needed: 1  trust model: pgp
gpg: depth: 0  valid:   1  signed:   0  trust: 0-, 0q, 0n, 0m, 0f, 1u


In [68]:
debdiff ../cmatrix_2.0-3.dsc ../cmatrix_2.0-3ubuntu1.dsc > ../1-2.0-3ubuntu1.debdiff

: 1